# Decision Trees

## Training and Visualizing a Decision Tree

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [2]:
X.shape

(150, 2)

In [3]:
y.shape

(150,)

In [5]:
from sklearn.tree import export_graphviz

export_graphviz(
        tree_clf,
        out_file="iris_tree.dot",
        feature_names=iris.feature_names[2:],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

In [6]:
! dot -Tpng iris_tree.dot -o iris_tree.png

![](iris_tree.png)

In particular, they don’t require feature scaling or centering at all. scaling the input features will just be a waste of time.

a node’s gini attribute measures its impurity: a node is “pure” (gini=0) if all training instances it applies to belong to the same class.

the depth-2 left node has a gini score equal to $1 – (0/54)^2 – (49/54)^2 – (5/54)^2 ≈ 0.168$

## Estimating Class Probabilities


In [8]:
tree_clf.predict_proba([[5, 1.5]])

array([[ 0.        ,  0.90740741,  0.09259259]])

In [9]:
tree_clf.predict([[5, 1.5]])

array([1])

Scikit-Learn uses the Classification And Regression Tree (CART) algorithm to train Decision Trees

Making predictions requires traversing the Decision Tree from the root to a leaf.

In [10]:
tree_clf = DecisionTreeClassifier(max_depth=2, presort=True)
# Scikit-Learn can speed up training by presorting the data (set presort=True), but this slows down training considerably for larger training sets
tree_clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=True, random_state=None,
            splitter='best')

In [11]:
tree_clf.predict_proba([[5, 1.5]])

array([[ 0.        ,  0.90740741,  0.09259259]])

In [12]:
tree_clf.predict([[5, 1.5]])

array([1])

the training algorithm compares all features (or less if `max_features` is set) on all samples at each node.

By default, the Gini impurity measure is used, but you can select the entropy impurity measure instead by setting the criterion hyperparameter to "entropy".

Should you use Gini impurity or entropy?

* Gini impurity is slightly faster to compute, tends to isolate the most frequent class in its own branch of the tree.
* Entropy tends to produce slightly more balanced trees

Reducing max_depth will regularize the model and thus reduce the risk of **overfitting**

 Increasing `min_*` hyperparameters or reducing `max_*` hyperparameters will regularize the model.

## Regression

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)
#The main difference is that instead of predicting a class in each node, it predicts a value.

The CART algorithm works mostly the same way as earlier, except that instead of trying to split the training set in a way that minimizes impurity, it now tries to split the training set in a way that minimizes the MSE

## Exercices

**1- What is the approximate depth of a Decision Tree trained (without restrictions) on a training set with 1 million instances?**

The depth of a well-balanced binary tree containing m leaves is equal to $log2(m)^2$, rounded up. if the training set contains one million instances, the Decision Tree will have a depth of $log2(106) ≈ 20$.

**5- If it takes one hour to train a Decision Tree on a training set containing 1 million instances, roughly how much time will it take to train another Decision Tree on a training set containing 10 million instances?**

The computational complexity of training a Decision Tree is  $O(n × m log(m))$. if you multiply the training set size by 10, the training time will be multiplied by $K = (n × 10m × log(10m)) / (n × m × log(m)) = 10 × log(10m) / log(m)$. If $m = 10^6$, then `K ≈ 11.7`, so you can expect the training time to be roughly 11.7 hours.

**6- If your training set contains 100,000 instances, will setting presort=True speed up training?**

Presorting the training set speeds up training only if the dataset is smaller than a few thousand instances. If it contains 100,000 instances, setting `presort=True` will considerably slow down training.

**7- Train and fine-tune a Decision Tree for the moons dataset.**

In [125]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=10000, noise=0.4)

In [149]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.2)

In [150]:
from sklearn.model_selection import GridSearchCV

In [151]:
param_grid = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}

des = DecisionTreeClassifier()

grid = GridSearchCV(des, param_grid=param_grid, cv=4)
result = grid.fit(X_train, y_train)

In [152]:
result.best_params_

{'max_leaf_nodes': 24, 'min_samples_split': 2}

In [153]:
result.best_score_

0.859375

In [154]:
grid.score(test_x, test_y)

0.86850000000000005

In [155]:
import pandas as pd

pd.DataFrame(result.cv_results_).T

/Users/hakan/.pyenv/versions/miniconda3-latest/lib/python3.5/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/hakan/.pyenv/versions/miniconda3-latest/lib/python3.5/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/hakan/.pyenv/versions/miniconda3-latest/lib/python3.5/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  wa

,0,1,2,3,4,5,6,7,8,9,...,284,285,286,287,288,289,290,291,292,293
mean_fit_time,0.00339204,0.00342476,0.00389224,0.00437468,0.00418496,0.00379205,0.00424951,0.00414824,0.00420153,0.00483829,...,0.0131614,0.014876,0.0126915,0.0113176,0.0092935,0.00909376,0.00997627,0.00998843,0.00947696,0.00973105
mean_score_time,0.000525236,0.000918746,0.00108057,0.000637293,0.000501275,0.00050652,0.000491321,0.000476182,0.000506222,0.000504732,...,0.00150633,0.000652254,0.00140995,0.00116128,0.000618696,0.000583291,0.000745714,0.000813842,0.000817597,0.000677943
mean_test_score,0.767125,0.767125,0.767125,0.8175,0.8175,0.8175,0.853125,0.853125,0.853125,0.853125,...,0.849625,0.84925,0.849625,0.849625,0.849625,0.849,0.849,0.8495,0.8495,0.849375
mean_train_score,0.773208,0.773208,0.773208,0.822166,0.822166,0.822166,0.858542,0.858542,0.858542,0.858542,...,0.891625,0.891708,0.891708,0.891791,0.892041,0.892,0.891875,0.892333,0.892333,0.892208
param_max_leaf_nodes,2,2,2,3,3,3,4,4,4,5,...,96,97,97,97,98,98,98,99,99,99
param_min_samples_split,2,3,4,2,3,4,2,3,4,2,...,4,2,3,4,2,3,4,2,3,4
params,"{'min_samples_split': 2, 'max_leaf_nodes': 2}","{'min_samples_split': 3, 'max_leaf_nodes': 2}","{'min_samples_split': 4, 'max_leaf_nodes': 2}","{'min_samples_split': 2, 'max_leaf_nodes': 3}","{'min_samples_split': 3, 'max_leaf_nodes': 3}","{'min_samples_split': 4, 'max_leaf_nodes': 3}","{'min_samples_split': 2, 'max_leaf_nodes': 4}","{'min_samples_split': 3, 'max_leaf_nodes': 4}","{'min_samples_split': 4, 'max_leaf_nodes': 4}","{'min_samples_split': 2, 'max_leaf_nodes': 5}",...,"{'min_samples_split': 4, 'max_leaf_nodes': 96}","{'min_samples_split': 2, 'max_leaf_nodes': 97}","{'min_samples_split': 3, 'max_leaf_nodes': 97}","{'min_samples_split': 4, 'max_leaf_nodes': 97}","{'min_samples_split': 2, 'max_leaf_nodes': 98}","{'min_samples_split': 3, 'max_leaf_nodes': 98}","{'min_samples_split': 4, 'max_leaf_nodes': 98}","{'min_samples_split': 2, 'max_leaf_nodes': 99}","{'min_samples_split': 3, 'max_leaf_nodes': 99}","{'min_samples_split': 4, 'max_leaf_nodes': 99}"
rank_test_score,292,292,292,289,289,289,160,160,160,160,...,254,280,254,254,254,286,286,265,265,276
split0_test_score,0.771614,0.771614,0.771614,0.821589,0.821589,0.821589,0.858071,0.858071,0.858071,0.858071,...,0.857571,0.856072,0.857571,0.857571,0.857571,0.856072,0.856072,0.857571,0.857571,0.857571
split0_train_score,0.772462,0.772462,0.772462,0.816303,0.816303,0.816303,0.858143,0.858143,0.858143,0.858143,...,0.889982,0.889982,0.889982,0.889982,0.890148,0.890148,0.890148,0.890815,0.890815,0.890815


In [45]:
result.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=31,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=3,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

**8- Grow a forest.**



a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn's ShuffleSplit class for this.

In [77]:
X_train.shape

(8000, 2)

In [132]:
y_train.shape

(8000,)

In [161]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    #  to make train size 100, test size 2215!!!
    mini_sets.append((X_mini_train, y_mini_train))

In [162]:
# test size!
len(X_train) - n_instances

7900

In [163]:
len(mini_sets)

1000

In [164]:
mini_sets[0][0].shape, mini_sets[0][1].shape

((100, 2), (100,))

In [165]:
from sklearn.metrics import accuracy_score
import numpy as np

b. Train one Decision Tree on each subset, using the best hyperparameter values found above. Evaluate these 1,000 Decision Trees on the test set. Since they were trained on smaller sets, these Decision Trees will likely perform worse than the first Decision Tree, achieving only about 80% accuracy.

In [166]:
from sklearn.base import clone

forest = [clone(result.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    
    # fit the decision tree with the best params
    tree.fit(X_mini_train, y_mini_train)
    
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

    
np.mean(accuracy_scores)

0.79209000000000007

c. Now comes the magic. For each test set instance, generate the predictions of the 1,000 Decision Trees, and keep only the most frequent prediction (you can use SciPy's mode() function for this). This gives you majority-vote predictions over the test set.

In [167]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)

In [168]:
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

d. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a Random Forest classifier!

In [94]:
test_y.shape

(2000,)

In [95]:
y_pred_majority_votes.shape

(1, 2000)

In [100]:
y_pred_majority_votes.reshape([-1,1]).shape

(2000, 1)

In [101]:
y_pred_majority_votes.reshape([-1]).shape

(2000,)

In [102]:
accuracy_score(test_y, y_pred_majority_votes.reshape([-1]))

0.87250000000000005

In [105]:
# predicted result!
y_pred_majority_votes

array([[1, 0, 1, ..., 1, 1, 1]], dtype=uint8)